# Reinforcement Learning am Beispiel von Tic Tac Toe #

https://www.python-lernen.de/tic-tac-toe-kontrolle-spielzug.htm enthält eine Implementierung mit einer Zufallsstrategie für die KI auf folgender Unterseite:<br>
https://www.python-lernen.de/tic-tac-toe-ki.htm



## Import von Bibliotheken ##

In [1]:
import numpy as np
from collections import defaultdict
import graphviz

In [2]:
class TreeNode:
    def __init__(self, state, player=None, parent=None):
        self.state = state
        self.player = player
        self.parent = parent  # Referenz auf den Elternknoten
        self.children = {}
        self.q_values = {}  # Q-Werte für Aktionen in diesem Zustand

    def add_child(self, action_index, next_state):
        if action_index not in self.children:
            self.children[action_index] = next_state

    def get_child(self, action_index):
        return self.children.get(action_index)
    
    def get_child_action(self, child):
        for action, state in self.children.items():
            if state == child:
                return action
        return None  # gib None zurück, falls child nicht existiert


    def set_q_value(self, action_index, q_value):
        self.q_values[action_index] = q_value

    def get_q_value(self, action_index):
        return self.q_values.get(action_index)
    
    def get_q_value_action(self, max_q_value):
        for action, q_value in self.q_values.items():
            if q_value == max_q_value:
                return action
        return None  # gib None zurück, falls child nicht existiert
    
    def visualize_tree(self, filename='tree'):
        dot = graphviz.Digraph(comment='Game Tree')
        self._add_nodes_edges(dot)
        dot.render(filename, format='png', cleanup=True)

    def _add_nodes_edges(self, dot):
        dot.node(str(id(self)), str(self.state))
        queue = [self]
        while queue:
            node = queue.pop(0)
            for action, child in node.children.items():
                dot.node(str(id(child)), str(child.state))
                color = "blue" if child.player == 1 else "green"
                dot.edge(str(id(node)), str(id(child)), label=(str(action)+" | "+str(node.get_q_value(action))), color=color)
                queue.append(child)
                
    def to_graphviz(self) -> graphviz.Digraph:
        """
        Transform the current fst into a graphviz graph

        Returns
        -------
        graph :  graphviz.DiGraph
            A graphviz DiGraph representing the fst

        """
        graph = graphviz.Digraph(format='svg')
        graph.node(str(id(self)), str(self.state))
        queue = [self]
        while queue:
            node = queue.pop(0)
            for action, child in node.children.items():
                graph.node(str(id(child)), str(child.state))
                color = "blue" if child.player == 1 else "green"
                graph.edge(str(id(node)), str(id(child)), label=(str(action)+" | "+str(node.get_q_value(action))), color=color)
                queue.append(child)       
        return graph
     

# Hauptfunktion zum Spielen von Tic-Tac-Toe und Training des Q-Baums
def play_tic_tac_toe(root_node, num_episodes=1000, epsilon=0.1, alpha=0.5, gamma=0.99):
    for _ in range(num_episodes):
        state = root_node
        player = 1
        for _ in range(9):  # Maximal 9 Züge im Tic-Tac-Toe
            # Choose action based on epsilon-greedy policy
            action = choose_action(state, epsilon, player)
            # Update the game state for player X=1
            next_state, done = update_state(state, action, player, alpha, gamma)
            # If the game is finished, break the loop
            if done:
                break
            # If the game is not finished, switch to the opponent's turn (player O=-1)
            state = next_state
            player = -1 * player
            
            
def choose_action(state, epsilon, player):
    if np.random.uniform(0, 1) < epsilon or not state.q_values:
#        return random_strategy(state, player)
        return heuristic_strategy(state, player)
    else:
        current_max_q_value = max([player * v for v in state.q_values.values()])
        if current_max_q_value <= 0:
            return random_strategy_unique(state, player) 
#            return heuristic_strategy(state, player)
        else:
            return state.get_q_value_action(player * current_max_q_value)
    
def random_strategy(state, player):
    return np.random.choice([i for i, val in enumerate(state.state) if val is None])

def random_strategy_unique(state, player):
    return np.random.choice([i for i, val in enumerate(state.state) if val is None and i is not state.q_values.keys()])


def heuristic_strategy(state, player):
    directions = np.array([
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Horizontal
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Vertikal
        [0, 4, 8], [2, 4, 6]              # Diagonal
    ])
    action_result = defaultdict(list)
    for i, val in enumerate(state.state):
        if val is None:
            prio = 0
            relevant = directions[np.where(np.any(directions == i, axis=1))]
            relevant = np.reshape(relevant[relevant != i],(-1, 2))
            for entry in np.take(state.state, relevant):
                if np.all(entry != None):
                    # print("Entry not None: " + str(entry))
                    # print("Player: " + str(player))
                    val = sum(entry) * player
                    if val == 2:
                        return i
                    elif val == -2:
                        prio = 9
                elif np.all(entry is None):
                    prio += 1
                elif np.any(entry == player):
                    prio += 3
                else:
                    prio += 1
            action_result[prio].append(i)
    return np.random.choice(action_result[max(action_result.keys())])
                        
                
    
#def q_value_strategy(state, player):
#    current_max_q_value = max([player * v for v in state.q_values.values()])
#    if current_max_q_value < 0:
#        random_strategy(state, player)
#    else:
#        return state.get_q_value_action(player * current_max_q_value) 

def update_state(state, action, player, alpha, gamma):
    next_state = state.get_child(action)  # Überprüfen, ob ein Kindknoten für diese Aktion existiert
    if next_state is None:
        next_state = TreeNode(state.state.copy(), player, state)  # Kopieren des aktuellen Zustands
        # print("Action: "+str(action))
        next_state.state[action] = player
        # print("next_state: " + str(next_state.state))
        state.add_child(action, next_state)  # Neuen Zustand als Kindknoten hinzufügen
        done = check_winner(next_state.state, player)
        if done:
            state.set_q_value(action, player)
            # print("Q_Values: " + str(state.q_values))
            new_q_value = max([player * v for v in state.q_values.values()])
            # print("New_Q_Value: " + str(new_q_value))
            update_q_values(state.parent, state, new_q_value * player, -1*player, alpha, gamma)
        else:
            state.set_q_value(action, 0)  # Eintragen des Rewards in die Q-Werte
    else:
        reward = state.get_q_value(action)
        done = ((player * reward) == 1)

    return next_state, done


def check_winner(state, player):
    win_patterns = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Horizontal
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Vertikal
        [0, 4, 8], [2, 4, 6]               # Diagonal
    ]
    for pattern in win_patterns:
        if all(state[pos] == player for pos in pattern):
            return True
    return False

def update_q_values(state, child, next_max_q_value, player, alpha, gamma):
    if not state is None:
        action = state.get_child_action(child)  # Aktion finden, die zum aktuellen Zustand geführt hat
        # print("Action: "+ str(action))
        old_q_value = state.get_q_value(action)
        if old_q_value != next_max_q_value:
            state.set_q_value(action, old_q_value + alpha * (gamma * next_max_q_value - old_q_value))
            new_q_value = max([player * v for v in state.q_values.values()])
            # print("New_Q_Value: "+ str(new_q_value))
            update_q_values(state.parent, state, new_q_value * player, -1*player, alpha, gamma)
    
if __name__ == "__main__":
    root_node = TreeNode([None] * 9)
    play_tic_tac_toe(root_node, 50)
    root_node.visualize_tree('tic_tac_toe_tree')
    # tree = root_node.to_graphviz()
    
# tree


## Verfeinerung der Implementierung - Übergabe von Spielstrategien als Parameter ##

Zunächst ist es wichtig, dass verschiedene Spielstrategien als Argument der Hauptfunktion übergeben werden. Dazu gehört - ähnlich der Othello-Implementierung in Lisp - die Umsetzung einer Humen-Strategie, um tatsächlich das Spiel gegen einen menschlichen Spieler zu spielen. Andy Schleising hat in seiner Bachelorarbeit eine entsprechende Implementierung vorgenommen, die es hier zu integrieren gilt.

This decorator is useful for optimizing the performance of recursive or expensive functions, as it caches the results of previous calls and returns them if the same arguments are passed again. It also uses the functools.wraps decorator to preserve the name and docstring of the original function. Here is the code:

## Verfeinerung der Implementierung - Vorhalten einer Hashtable für alle bereits erkundeten Zustände ##

Bei verschiedenen kombinatorischen Spielen inkl. Tic Tac Toe kann es zu gleichen Spielzuständen (Game-State) kommen, obwohl die Reihenfolge der Spielzüge (Aktionen) unterschiedlich war (Play-State). Deshalb wäre es wichtig, immer wenn im Spielbaum ein neuer Spielzustand untersucht wird, ersteinmal zu schauen, ob man diesen Spielzustand bereits untersucht hat. Hierfür eignet sich das Konzept des Memoization. Für Python gibt es mittlerweile dieses Konzept als Decorator (vgl. https://medium.com/@ayush-thakur02/python-decorators-that-can-reduce-your-code-by-half-b19f673bc7d8). Decoratoren gibt es auch für andere Konzepte, die sich nachträglich in den Programmcode einklinken. Insgesamt werden die Decoratoren: @measure_time, @wrapper, @timer, @debug und @memoize vorgestellt sowie auf die Decoratoren @wraps, @staticmethod, @classmethod, @property, @functools.lru_cache, @functools.singledispatch verwiesen. Hier interessiert zunächst aber nur das Memoization. Deshalb folgt auf dem genannten Artikel der entsprechende Pythonauszug. Jedoch muss man für die Implementierung im vorliegenden Reinforcement-Szenario aufpassen, weil nur die Berechnung des Funktionswertes optimiert wird, d.h. nur bei gleichen Argumenten spart man sich die Berechnung.

In [3]:
from functools import wraps

def memoize(func):
    cache = {}
    @wraps(func)
    def wrapper(*args):
        if args in cache:
            return cache[args]
        else:
            result = func(*args)
            cache[args] = result
            return result
    return wrapper

Now, you can use this decorator to memoize any function, such as:

In [ ]:
@memoize
def factorial(n):
    """Returns the factorial of n"""
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)
@memoize
def fibonacci(n):
    """Returns the nth Fibonacci number"""
    if n == 0 or n == 1:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
print(factorial(10))
print(fibonacci(10))

This would output the same as before, but with much faster execution time, as the results are cached and reused.